## Loading the libraries

In [1]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


## Loading API Keys

In [2]:
load_dotenv()

True

In [3]:
## load the Cohere API key
os.environ['COHERE_API_KEY']=os.getenv("COHERE_API_KEY")

## load the Groq API key
groq_api_key=os.environ['GROQ_API_KEY']

## Data Ingestion

In [4]:
loader = PyPDFLoader('ecofriendly_dataset_updated.pdf')
documents = loader.load()

In [5]:
documents

[Document(metadata={'source': 'ecofriendly_dataset_updated.pdf', 'page': 0}, page_content='Engine (Internal Combustion)  \nEngine Block  \n• Material : Recycled aluminum, bio -based plastics for engine covers.  \n• Eco-Friendliness : Recycled aluminum reduces waste and conserves natural resources. \nBio-based plastics are biodegradable and reduce dependency on petroleum -based plastics.  \nCylinder Head  \n• Material : Recycled aluminum, ceramic coatings for heat management.  \n• Eco-Friendliness : Recycled aluminum conserves resources. Ceramic coatings extend the \nlifespan of components, reducing waste.  \nPistons  \n• Material : Lightweight aluminum alloys, ceramic coatings for heat resistance.  \n• Eco-Friendliness : Lightweight materials improve fuel efficiency, reducing emissions.  \nConnecting Rods  \n• Material : High -strength steel alloys with reduced carbon footprint, or lightweight \ntitanium alloys.  \n• Eco-Friendliness : Reduced carbon footprint in production processes, 

## Chunking the documents

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
final_document = text_splitter.split_documents(documents)

In [7]:
final_document

[Document(metadata={'source': 'ecofriendly_dataset_updated.pdf', 'page': 0}, page_content='Engine (Internal Combustion)  \nEngine Block  \n• Material : Recycled aluminum, bio -based plastics for engine covers.  \n• Eco-Friendliness : Recycled aluminum reduces waste and conserves natural resources. \nBio-based plastics are biodegradable and reduce dependency on petroleum -based plastics.  \nCylinder Head  \n• Material : Recycled aluminum, ceramic coatings for heat management.  \n• Eco-Friendliness : Recycled aluminum conserves resources. Ceramic coatings extend the \nlifespan of components, reducing waste.  \nPistons  \n• Material : Lightweight aluminum alloys, ceramic coatings for heat resistance.  \n• Eco-Friendliness : Lightweight materials improve fuel efficiency, reducing emissions.  \nConnecting Rods'),
 Document(metadata={'source': 'ecofriendly_dataset_updated.pdf', 'page': 0}, page_content='Pistons  \n• Material : Lightweight aluminum alloys, ceramic coatings for heat resistance

## Vector Embeddings

In [9]:
vector_db = FAISS.from_documents(final_document, CohereEmbeddings())

In [34]:
# Designing the prompt template
prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant.
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer but don't mention "Based on the context". 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [35]:
# Loading the llm models
llm = ChatGroq(groq_api_key = groq_api_key,
               model_name = "mixtral-8x7b-32768"
               )

## Chain and Retrieval

In [36]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [37]:
retriever = vector_db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

## Query that can be asked !!!

**Query**
 - Recommend a sustainable material for the engine block of an internal combustion engine that prioritizes reducing waste and conserving natural resources.
 - What material should be used for the cylinder head to enhance heat management and conserve resources?
 - Suggest an eco-friendly material for pistons that improves fuel efficiency and reduces emissions.
 -  Identify a sustainable material for connecting rods with a focus on reducing the carbon footprint during production.
 - Which material would be best for the crankshaft to optimize weight reduction and improve fuel efficiency?
 - Recommend an environmentally friendly material for gear housing in the transmission system.
 - What advanced materials should be used for gears to ensure durability and weight reduction?
 - Suggest organic materials for clutch plates that have a lower environmental impact.
 -  Recommend a material for the driveshaft that is lightweight and improves fuel efficiency.
 - What materials should be used for the electric motor to reduce environmental impact while maintaining performance?
 - Identify an eco-friendly battery material with recycled components for electric vehicles.
 - Suggest materials for power electronics that enhance efficiency and reduce energy consumption.
 - What ceramic material would be suitable for components requiring high thermal conductivity and wear resistance?
 - Recommend a ceramic material with excellent thermal shock resistance and fracture toughness.
 - Suggest a material for axles that optimizes weight reduction and improves fuel efficiency.

## Generating Responses

In [38]:
query = "What ceramic material would be suitable for components requiring high thermal conductivity and wear resistance?"
response = retrieval_chain.invoke({
    "input" : query
})
print(response['answer'])

To determine the suitable ceramic material for components requiring high thermal conductivity and wear resistance, let's consider the properties of each material mentioned in the context:

1. Alumina (Al2O3)
   - High strength
   - High hardness
   - High thermal conductivity

2. Zirconia (ZrO2)
   - Excellent thermal shock resistance
   - High fracture toughness

3. Silicon Carbide (SiC)
   - Known for its high thermal conductivity
   - High wear resistance

Given these properties, Silicon Carbide (SiC) appears to be the most suitable ceramic material for components requiring high thermal conductivity and wear resistance. It is specifically known for these properties, making it an ideal choice for such applications.


In [39]:
query_1 = "Suggest an eco-friendly material for pistons that improves fuel efficiency and reduces emissions."
response1 = retrieval_chain.invoke({
    "input" : query_1
})
print(response1['answer'])

For pistons, an eco-friendly material that improves fuel efficiency and reduces emissions is lightweight aluminum alloys. This material is already used in piston manufacturing, as stated in the context, and it contributes to enhanced fuel efficiency due to its lighter weight. This reduced weight requires less energy to move, which in turn decreases emissions from the engine.


In [40]:
query_2 = "Recommend an environmentally friendly material for gear housing in the transmission system."
response2 = retrieval_chain.invoke({
    "input" : query_2
})
print(response2['answer'])

For an environmentally friendly material for the gear housing in the transmission system, recycled aluminum or magnesium alloys would be ideal. These materials reduce waste and resource extraction, contributing to a more sustainable and eco-friendly production process.


In [41]:
query_3 = "Identify an eco-friendly battery material with recycled components for electric vehicles."
response3 = retrieval_chain.invoke({
    "input" : query_3
})
print(response3['answer'])

The battery material for electric vehicles, as mentioned in the context, is lithium-ion batteries. These batteries incorporate recycled components which contribute to their eco-friendliness. The recycling of components reduces waste and enhances the sustainability of electric vehicle production.


In [42]:
query_4 = "Recommend a sustainable material for the engine block of an internal combustion engine that prioritizes reducing waste and conserving natural resources."
response4 = retrieval_chain.invoke({
    "input" : query_4
})
print(response4['answer'])

For the engine block of an internal combustion engine, recycled aluminum would be a sustainable material that prioritizes reducing waste and conserving natural resources. This is because recycled aluminum reduces waste by reusing materials that would otherwise be discarded, and it conserves natural resources by decreasing the need for mining and processing new aluminum. Additionally, aluminum is a lightweight material, which can contribute to improved fuel efficiency in vehicles, further reducing their environmental impact.


In [44]:
query_5 = "Suggest a material for axles that optimizes weight reduction and improves fuel efficiency."
response5 = retrieval_chain.invoke({
    "input" : query_5
})
print(response5['answer'])

The material for axles that optimizes weight reduction and improves fuel efficiency is high-strength steel alloys with an optimized design for weight reduction. This is based on the provided context, as it mentions high-strength steel alloys are used for axles in both traditional and electric vehicles, and this material choice contributes to weight reduction and improved fuel efficiency due to the optimized design.


<h2 align="center">Thank You !!! </h2>